In [863]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.polynomial.legendre as leg

Set up classes

In [864]:
class particle:
     def __init__(self, alive, mu, x, g, w):
        self.alive = alive
        self.mu = mu
        self.x = x
        self.g = g
        self.w = w

In [865]:
class source:
    def __init__(self, scatter, fission):
        self.scatter = scatter
        self.fission = fission

In [866]:
class tally:
    def __init__(self, scalar_flux, current, k):
        self.scalar_flux = scalar_flux
        self.current = current
        self.k = k
    
    def zero(self):
        self.scalar_flux *= 0
        self.current *= 0

In [867]:
class material:
    def __init__(self, nuSigmaF, SigmaS, SigmaT):
        self.nuSigmaF = nuSigmaF
        self.SigmaS = SigmaS
        self.SigmaT = SigmaT

In [868]:
class cell:
    def __init__(self, left, right, material):
        self.left = left
        self.right = right
        self.material = material

In [869]:
class mesh:
    def __init__(self, Nx, L, bc, U, W):
        self.Nx = Nx
        self.L = L
        self.dx = L/(Nx-1)
        self.mesh_x = np.linspace(0, L, Nx)
        self.mesh_flux = np.linspace(0, L, Nx-1)
        self.bc = bc
        self.U = U
        self.W = W

In [870]:
class bc:
    def __init__(self, bc):
        self.bc = bc

Set up functions

In [871]:
def zero_tally(tally):
    tally.scalar_flux = np.zeros_like(tally.scalar_flux)
    tally.current = np.zeros_like(tally.current)

In [872]:
def f_convergence(new, old):
    # convergence criteria
    epi = 1e-5
    if np.amax(abs(new - old)) < epi:
        return True
    else:
        return False

In [873]:
def k_convergence(new, old):
    # convergence criteria
    ek = 1e-5
    if np.amax(abs((new - old) / new)) < ek:
        return True
    else:
        return False

In [874]:
def si_convergence(new, old):
    # convergence criteria
    esi = 1e-6
    if np.amax(abs(new - old) / new) < esi:
        return True
    else:
        return False

In [875]:
def get_surface(p, bin_id):
    if p.mu > 0:
        return bin_id+1
    else:
        return bin_id

In [876]:
def get_material(p, cells):
    for i in range(len(cells)):
        right = cells[i].right
        left = cells[i].left
        if p.x >= left and p.x < right:
            return cells[i].material

In [877]:
def calculate_fission_source():
    # get values
    flux = tally1.scalar_flux
    source = np.zeros(mesh1.Nx-1)
    dx = mesh1.dx
    index = 0
    
    # loop through all cells
    for i in range(len(cells)):
        # get cell and material data
        left = cells[i].left
        right = cells[i].right
        material = cells[i].material
        nuSigmaF = material.nuSigmaF
        bins = int((right - left) / dx)
        
        # calculate source for current cell
        for j in range(bins):
            j += index
            source[j] = nuSigmaF[0]*flux[j,0] + nuSigmaF[1]*flux[j,1]
            
        index = j+1
        
    return source

In [878]:
def calculate_k(old):
    dx = mesh1.dx
    k_new = tally1.k * np.sum(dx*source1.fission)/np.sum(dx*old)
    return k_new

In [879]:
def calculate_source_S(g):
    # get values
    Sg = 1/tally1.k * source1.fission
    scalar_flux = tally1.scalar_flux[:,g]
    dx = mesh1.dx
    index = 0
    if g == 0:
        g_prime = 1
    else:
        g_prime = 0
    
    # loop through all cells
    for i in range(len(cells)):
        # get cell and material data
        left = cells[i].left
        right = cells[i].right
        material = cells[i].material
        SigmaS = material.SigmaS
        bins = int((right - left) / dx)
        
        # calculate source for current cell
        for j in range(bins):
            j += index
            Sg[j] += SigmaS[g, g_prime] * scalar_flux[j]
        index = j+1

    return Sg

In [880]:
def calculate_source_Q(S, g):
    # get values
    Q = S.copy()
    scalar_flux = tally1.scalar_flux[:,g]
    dx = mesh1.dx
    index = 0

    # loop through all cells
    for i in range(len(cells)):
        # get cell and material data
        left = cells[i].left
        right = cells[i].right
        material = cells[i].material
        SigmaS = material.SigmaS
        bins = int((right - left) / dx)
        
        # calculate source for current cell
        for j in range(bins):
            j += index
            Q[j] += SigmaS[g, g] * scalar_flux[j]
        index = j+1

    return Q 

In [881]:
def cell_flux(Q, SigmaT, dx, u, psi):
    tau = SigmaT * dx / abs(u)
    cell_flux = psi * np.exp(-tau) + Q/SigmaT * (1 - np.exp(-tau))
    return cell_flux

In [882]:
def sweep(bc_incoming, Q, n, g):
    u = mesh1.U[n]
    w = mesh1.W[n]
    angular_flux_old = bc_incoming

    # sweep right
    if u > 0:
        index = 0
        # loop through all cells
        for i in range(len(cells)):
            # get cell and material data
            dx = mesh1.dx
            left = cells[i].left
            right = cells[i].right
            material = cells[i].material
            SigmaT = material.SigmaT[g]
            bins = int((right - left) / dx)

            # calculate source for current cell
            for j in range(bins):
                j += index
                # compute angular flux
                angular_flux_new = cell_flux(Q[j], SigmaT, dx, u, angular_flux_old)
                angular_flux_old = angular_flux_new

                # tally flux and current
                tally1.scalar_flux[j] += w * angular_flux_new
                tally1.current[j] += w * u * angular_flux_new

            index = j+1
    
    # sweep left
    if u < 0:
        index = mesh1.Nx - 2

        # loop through all cells
        for i in range(len(cells)-1,-1,-1):
            # get cell and material data
            dx = mesh1.dx
            left = cells[i].left
            right = cells[i].right
            material = cells[i].material
            SigmaT = material.SigmaT[g]
            bins = int((right - left) / dx)

            # calculate source for current cell
            for j in range(bins):
                j = index - j

                # compute angular flux sweep left
                angular_flux_new = cell_flux(Q[j], SigmaT, dx, u, angular_flux_old)
                angular_flux_old = angular_flux_new 

                # tally flux and current
                tally1.scalar_flux[j] += w * angular_flux_new
                tally1.current[j] += w * u * angular_flux_new

            index -= j-1

    return angular_flux_new

Main loops

In [883]:
def source_iteration():
    iter = 100
    scalar_flux_old = np.zeros_like(tally1.scalar_flux)
    G = np.size(tally1.scalar_flux, axis=1)

    for g in range(G):
        S = calculate_source_S(g)

        for i in range(iter):
            Q = calculate_source_Q(S, g)
            tally1.zero()

            # sweep all angles
            for n in range(len(mesh1.U)):
                bc_in = np.flip(range(len(mesh1.U)))[n]
                bc1.bc[g, n] = sweep(bc1.bc[g, bc_in], Q, n, g)
            
            # check convergence
            if si_convergence(tally1.scalar_flux, scalar_flux_old):
                break

            scalar_flux_old = tally1.scalar_flux.copy()
            #print("source", i)

In [884]:
def power_iteration():
    iter = 100
    k_old = 1
    f_old = np.ones_like(source1.fission)
    for i in range(iter):
        source_iteration()

        source1.fission = calculate_fission_source()
        tally1.k = calculate_k(k_old)

        if f_convergence(source1.fission, f_old) and k_convergence(tally1.k, k_old):
            break

        k_old = tally1.k.copy()
        f_old = source1.fission.copy()
        #print("power", i)


In [885]:
def main_loop():
    power_iteration()
    print(tally1.scalar_flux, tally1.k)

Test

In [886]:
# materials
mox = material(np.array([0,1.05]), 
              np.array([[0.25,0],[0,0]]), 
              np.array([0.25,0.75]))
# cells
cells = [cell(0, 10, mox)]

# gauss-legendre
U, W = leg.leggauss(2)

# mesh
mesh1 = mesh(10 + 1, 10, "reflective", U, W)

# initialize tally
tally1 = tally(np.zeros((mesh1.Nx-1, 2)), 
            np.zeros((mesh1.Nx, 2)),
            1)

# initialize source
source1 = source(np.zeros_like(mesh1.mesh_flux), 
                np.ones_like(mesh1.mesh_flux))

# initialize bc
bc1 = bc(np.zeros((2, len(mesh1.U))))

In [887]:
main_loop()

[[0.26666667 0.26666667]
 [0.26666667 0.26666667]
 [0.26666667 0.26666667]
 [0.26666667 0.26666667]
 [0.26666667 0.26666667]
 [0.26666667 0.26666667]
 [0.26666667 0.26666667]
 [0.26666667 0.26666667]
 [0.26666667 0.26666667]
 [0.26666667 0.26666667]] 2.800000000000001


Problem A

In [888]:
# materials
mox = material(np.array([0,1.05]), 
              np.array([[0.25,0],[0,0]]), 
              np.array([0.25,0.75]))

u = material(np.array([0,0.40]), 
              np.array([[0.25,0],[0,0]]), 
              np.array([0.25,0.25]))

water = material(np.array([0,0]), 
              np.array([[0.20,0.05],[0,1.25]]), 
              np.array([0.25,1.25]))
# cells
cells = []

for i in range(8):
    i *= 1.25
    cells.extend([cell(i+0, i+.3125, water), cell(i+.3125, i+.9375, mox), cell(i+.9375, i+1.25, water)])

for i in range(8, 16):
    i *= 1.25
    cells.extend([cell(i+0, i+.3125, water), cell(i+.3125, i+.9375, u), cell(i+.9375, i+1.25, water)])

# gauss-legendre
U, W = leg.leggauss(10)

# mesh
mesh1 = mesh(128 + 1, 20, "reflective", U, W)

# tally
tally1 = tally(np.zeros((mesh1.Nx-1, 2)), 
               np.zeros((mesh1.Nx, 2)),
               1)